In [1]:
import pandas as pd
import os


DATAPATH = "../data"

In [12]:
df1 = pd.read_csv(os.path.join(DATAPATH, "original", "250425_sheet1.csv"))
df2 = pd.read_csv(os.path.join(DATAPATH, "original", "250425_sheet2.csv"))

In [13]:
print(df1.shape, df2.shape)
df1 = df1[~df1["Smiles"].isna()]
df2 = df2[~df2["Smiles"].isna()]
print(df1.shape, df2.shape)

(513, 21) (467, 18)
(490, 21) (465, 18)


In [14]:
smi1 = df1["Smiles"]
smi2 = df2["Smiles"]

In [15]:
print(len(smi1), len(smi2))

490 465


In [16]:
smi1[4]

'C=CCSSCC=C'

In [21]:
from rdkit import Chem
inchi1 = []
for smi in smi1:
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        print("Invalid SMILES:", smi)
    else:
        inchikey = Chem.MolToInchiKey(mol)
        inchi1.append(inchikey)

inchi2 = []
for smi in smi2:
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        print("Invalid SMILES:", smi)
    else:
        inchikey = Chem.MolToInchiKey(mol)
        inchi2.append(inchikey)

print(len(inchi1), len(inchi2))

[17:21:42] SMILES Parse Error: syntax error while parsing: C1=C(C=C(C(=C1O)O)O)C(=O)OC[C@@H]2[C@H]([C@@H]([C@H([C@@H(O2)OC(=O)C3=CC(=C(C(=C3)O)O)O)OC(=O)C4=CC(=C(C(=C4)O)O)O)OC(=O)C5=CC(=C(C(=C5)O)O)O)OC(=O)C6=CC(=C(C(=C6)O)O)O
[17:21:42] SMILES Parse Error: Failed parsing SMILES 'C1=C(C=C(C(=C1O)O)O)C(=O)OC[C@@H]2[C@H]([C@@H]([C@H([C@@H(O2)OC(=O)C3=CC(=C(C(=C3)O)O)O)OC(=O)C4=CC(=C(C(=C4)O)O)O)OC(=O)C5=CC(=C(C(=C5)O)O)O)OC(=O)C6=CC(=C(C(=C6)O)O)O' for input: 'C1=C(C=C(C(=C1O)O)O)C(=O)OC[C@@H]2[C@H]([C@@H]([C@H([C@@H(O2)OC(=O)C3=CC(=C(C(=C3)O)O)O)OC(=O)C4=CC(=C(C(=C4)O)O)O)OC(=O)C5=CC(=C(C(=C5)O)O)O)OC(=O)C6=CC(=C(C(=C6)O)O)O'
[17:21:42] SMILES Parse Error: syntax error while parsing: CSCPPACGZOOCGX-UHFFFAOYSA-N
[17:21:42] SMILES Parse Error: Failed parsing SMILES 'CSCPPACGZOOCGX-UHFFFAOYSA-N' for input: 'CSCPPACGZOOCGX-UHFFFAOYSA-N'
[17:21:42] SMILES Parse Error: syntax error while parsing: 1[C@H](C([C@](C(C1(C(=O)O)O)(C(=O)C2=CC(=C(C(=C2)O)O)O)C(=O)C3=CC(=C(C(=C3)O)O)O)(C(=O)C4=CC(=C(

Invalid SMILES: C1=C(C=C(C(=C1O)O)O)C(=O)OC[C@@H]2[C@H]([C@@H]([C@H([C@@H(O2)OC(=O)C3=CC(=C(C(=C3)O)O)O)OC(=O)C4=CC(=C(C(=C4)O)O)O)OC(=O)C5=CC(=C(C(=C5)O)O)O)OC(=O)C6=CC(=C(C(=C6)O)O)O
Invalid SMILES: CSCPPACGZOOCGX-UHFFFAOYSA-N
Invalid SMILES: 1[C@H](C([C@](C(C1(C(=O)O)O)(C(=O)C2=CC(=C(C(=C2)O)O)O)C(=O)C3=CC(=C(C(=C3)O)O)O)(C(=O)C4=CC(=C(C(=C4)O)O)O)O)(C(=O)C5=CC(=C(C(=C5)O)O)O)O)O
Invalid SMILES: CC(N=C[23])=NC(C(F)(F)F)=C@23CCC(N(@23)CC[6]=CN=C(C[13]()=CC=CC=C@13C[22]=NN=NN@22)C=C@7)=O
Invalid SMILES: O=C(C[3:S=I]HN(CCC@4)C(C[S=N]H(NC[S=N]H(CCC[16]=CC=CC=C@16)C(O)=O)C)=O)O
Invalid SMILES: O=C(O)CN(C(C(C)CSC(C)=O)=O)C[6:S=N]HC[S=I](C[12])(H)CCC[S=I]@12(H)C@7
Invalid SMILES: O=C(O)CN(CCC[17]=CC=CO@17)C(C(C)CSC(C)=O)=O
Invalid SMILES: O=C(O)CN(C[17]=C(C)C=CC=C@17)C(C(C)CS)=C
Invalid SMILES: O=C(N[18]CCCC)C[16]=C(N=CC(F)=C@16)N(@18)CC(C=C[12])=CC=C@12C[5]=C(C[23]=NNN=N@23)C=CC=C@6
487 459


In [22]:
all_smi = list(set(inchi1+inchi2))
print(len(all_smi))

814
